In [1]:
import maboss
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import shutil
import sys
import yaml
import numpy as np
sys.path.append("/Users/emilieyu/endotehelial-masboss")

ipylab module is not installed, menus and toolbar are disabled.


In [2]:
from boolean_models.analysis import (
    compute_delta,
    classify_phenotype,
    save_df_to_csv
)

In [3]:

PROJECT_ROOT = Path("/Users/emilieyu/endotehelial-masboss/")
CONFIG_PATH = PROJECT_ROOT / "config" / "rho_sim_config.yaml"

with open(CONFIG_PATH, "r") as f:
    config = yaml.safe_load(f)

# Resolve Paths
MODELS_BND = PROJECT_ROOT / config['paths']['model_bnd']
MODELS_CFG = PROJECT_ROOT / config['paths']['model_cfg']
RESULTS_DIR = PROJECT_ROOT / config['paths']['results_base']

# Result subdirectories
RAW_DIR = RESULTS_DIR / config['paths']['subdirs']['raw']
PROCESSED_DIR = RESULTS_DIR / config['paths']['subdirs']['processed']
SS_DIR = RESULTS_DIR / config['paths']['subdirs']['steady_state']

# --------------------------------------------------
# Global variable
# --------------------------------------------------
EPS = config['analysis']['thresholds']['eps']
PERBS_DICT = config.get('perturbations') # get mutations directly from config
N_RUNS=100

In [4]:
base_model = maboss.load(str(MODELS_BND), str(MODELS_CFG))

In [5]:
sweep_cfg = config['sensitivity_analysis'][0]
param_name = sweep_cfg['parameter']

In [6]:
values = np.arange(sweep_cfg['range'][0], sweep_cfg['range'][1], sweep_cfg['steps'])
values

array([ 5., 15., 25., 35., 45.])

In [7]:
wt_results = []

for val in values:
    m = base_model.copy()
    m.update_parameters(**{param_name: val})
    print(m.param[param_name])

    res = m.run()
    mean_df = res.get_nodes_probtraj()
    ss = mean_df.tail(1).copy()
    ss['delta'] = ss['RhoA'] - ss['RhoC']
    ss['param_value'] = val
    ss['param_name'] = param_name
    wt_results.append(ss)

5.0
15.0
25.0
35.0
45.0


In [9]:
pd.concat(wt_results, ignore_index=True)

,DSP,TJP1,JCAD,RhoA,RhoC,delta,param_value,param_name
0,1.000000,1.000000,1.000000,0.687969,0.831144,-0.143175,5.0,$RhoA_amp
1,1.000001,1.000001,1.000001,0.868326,0.799690,0.068636,15.0,$RhoA_amp
2,1.000000,1.000000,1.000000,0.916093,0.779314,0.136779,25.0,$RhoA_amp
3,1.000000,1.000000,1.000000,0.936845,0.774825,0.162020,35.0,$RhoA_amp
4,1.000000,1.000000,1.000000,0.952324,0.783736,0.168588,45.0,$RhoA_amp


## Manual 100 Sim Runs

how is it possible that convergence is at the same probabilities for all simulations??

In [50]:
perb_dict = {}
phenotype_dict = {}
#perbs_ss = []

# Run simulations. 
for name, mutation in PERBS_DICT.items():
    print(f"DEBUG: Running scenario: {name}")

    # Create model of KO scenario
    m = base_model.copy()
    for node, state in mutation.items():
        m.mutate(node, state)


    all_runs = []
    for i in range(N_RUNS):
        res = m.run()
        prob_df = res.get_last_nodes_probtraj()
        prob_df["run"] = i
        all_runs.append(prob_df)

    # Combine all runs
    prob_df = pd.concat(all_runs, ignore_index=True)

    # Compute Rho balance
    balance_df = prob_df.copy()
    balance_df["delta"] = compute_delta(balance_df)

    perb_dict[name] = balance_df
    
    # phenotype_df = classify_phenotype(balance_df)
    # phenotype_dict[name] = phenotype_df
    #print(phenotype_df)

DEBUG: Running scenario: WT
DEBUG: Running scenario: DSP
DEBUG: Running scenario: TJP1
DEBUG: Running scenario: JCAD
DEBUG: Running scenario: DSP_JCAD
DEBUG: Running scenario: TJP1_JCAD


In [54]:
dsp = perb_dict['DSP']
dsp['delta'].unique()

array([0.668165])

## Ensemble Moment

In [5]:

res = base_model.run()
base_model.get_initial_state()
ensemble = maboss.Ensemble(str(PROJECT_ROOT / "data" / "models"), str(MODELS_CFG), simulations=100)
ensemble.param['thread_count'] = 1
ensemble.print_cfg()


thread_count = 1;
DSP.is_internal = FALSE;
TJP1.is_internal = FALSE;
JCAD.is_internal = FALSE;
RhoA.is_internal = FALSE;
RhoC.is_internal = FALSE;



In [16]:

e = ensemble.copy()
e.mutate('DSP', 'OFF')

In [19]:
e.mutations
res = e.run()
prob_df = res.get_nodes_probtraj()
prob_df

,DSP,TJP1,RhoA,JCAD,RhoC
0.0,1.000000,1.000000,0.354098,1.000000,0.358173
0.1,1.000001,1.000001,0.691923,1.000001,0.695999
0.2,1.000000,1.000000,0.781404,1.000000,0.781084
0.3,1.000000,1.000000,0.804508,1.000000,0.804743
0.4,1.000000,1.000000,0.804364,1.000000,0.808941
...,...,...,...,...,...
49.5,1.000000,1.000000,0.802588,1.000000,0.810467
49.6,1.000000,1.000000,0.801073,1.000000,0.810740
49.7,1.000000,1.000000,0.802616,1.000000,0.812007
49.8,1.000000,1.000000,0.806200,1.000000,0.809442


In [ ]:
ensemble_dict = {}
ss_dict = {}
for name, mutation in PERBS_DICT.items():
    print(f"DEBUG: Running scenario: {name}")

    # Create model of KO scenario
    e = ensemble.copy()
    for node, state in mutation.items():
        e.mutate(node, state)
        e.

    # Run MaBoSS
    res = e.run()
    prob_df = res.get_nodes_probtraj().rename_axis('t').reset_index()
    
    # Compute Rho balance
    balance_df = prob_df.copy()  
    balance_df["delta"] = compute_delta(balance_df)
    ensemble_dict[name] = balance_df

    

    ss_df = res.get_last_nodes_probtraj()
    ss_dict[name] = ss_df

DEBUG: Running scenario: WT
DEBUG: Running scenario: DSP
thread_count = 1;
DSP.is_internal = FALSE;
TJP1.is_internal = FALSE;
JCAD.is_internal = FALSE;
RhoA.is_internal = FALSE;
RhoC.is_internal = FALSE;

DEBUG: Running scenario: TJP1
thread_count = 1;
DSP.is_internal = FALSE;
TJP1.is_internal = FALSE;
JCAD.is_internal = FALSE;
RhoA.is_internal = FALSE;
RhoC.is_internal = FALSE;

DEBUG: Running scenario: JCAD
thread_count = 1;
DSP.is_internal = FALSE;
TJP1.is_internal = FALSE;
JCAD.is_internal = FALSE;
RhoA.is_internal = FALSE;
RhoC.is_internal = FALSE;

DEBUG: Running scenario: DSP_JCAD
thread_count = 1;
DSP.is_internal = FALSE;
TJP1.is_internal = FALSE;
JCAD.is_internal = FALSE;
RhoA.is_internal = FALSE;
RhoC.is_internal = FALSE;

thread_count = 1;
DSP.is_internal = FALSE;
TJP1.is_internal = FALSE;
JCAD.is_internal = FALSE;
RhoA.is_internal = FALSE;
RhoC.is_internal = FALSE;

DEBUG: Running scenario: TJP1_JCAD
thread_count = 1;
DSP.is_internal = FALSE;
TJP1.is_internal = FALSE;
JCAD.

In [43]:
ensemble_dict['TJP1']

,t,DSP,TJP1,RhoA,JCAD,RhoC,delta
0,0.0,1.000000,1.000000,0.354098,1.000000,0.358173,0.004075
1,0.1,1.000001,1.000001,0.691923,1.000001,0.695999,0.004076
2,0.2,1.000000,1.000000,0.781404,1.000000,0.781084,-0.000320
3,0.3,1.000000,1.000000,0.804508,1.000000,0.804743,0.000235
4,0.4,1.000000,1.000000,0.804364,1.000000,0.808941,0.004577
...,...,...,...,...,...,...,...
495,49.5,1.000000,1.000000,0.802588,1.000000,0.810467,0.007879
496,49.6,1.000000,1.000000,0.801073,1.000000,0.810740,0.009667
497,49.7,1.000000,1.000000,0.802616,1.000000,0.812007,0.009391
498,49.8,1.000000,1.000000,0.806200,1.000000,0.809442,0.003242


In [40]:
ss_dict

{'WT':               DSP      JCAD      RhoA      RhoC      TJP1
 49.9000  0.999999  0.999999  0.806665  0.814914  0.999999,
 'DSP':               DSP      JCAD      RhoA      RhoC      TJP1
 49.9000  0.999999  0.999999  0.806665  0.814914  0.999999,
 'TJP1':               DSP      JCAD      RhoA      RhoC      TJP1
 49.9000  0.999999  0.999999  0.806665  0.814914  0.999999,
 'JCAD':               DSP      JCAD      RhoA      RhoC      TJP1
 49.9000  0.999999  0.999999  0.806665  0.814914  0.999999,
 'DSP_JCAD':               DSP      JCAD      RhoA      RhoC      TJP1
 49.9000  0.999999  0.999999  0.806665  0.814914  0.999999,
 'TJP1_JCAD':               DSP      JCAD      RhoA      RhoC      TJP1
 49.9000  0.999999  0.999999  0.806665  0.814914  0.999999}